In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
from pathlib import Path

import spikeinterface.full as si
from spikeinterface.sortingcomponents.peak_detection import detect_peaks
from spikeinterface.sortingcomponents.peak_localization import localize_peaks

from myfigures_interactive import *
from params import *
from export_files import export_files

import ipywidgets as widgets

%matplotlib widget

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Recording path
core_path = Path('/nas/Neuropixel_Recordings/AreaX-LMAN/Imp_30_03_2022/Recordings/')
rec_name = 'Rec_31_03_2022_sleep_g0' # Main thing to change between recordings
rec_path = core_path / rec_name

# Output path
output_path = rec_path

# if not output_path.exists():
#     output_path.mkdir()

In [ ]:
# Read recording
rec_ap = si.read_spikeglx(rec_path, stream_id='imec0.ap')
rec_preprocessed = si.bandpass_filter(si.common_reference(rec_ap, reference='local', local_radius=(50, 100)), freq_min=300, freq_max=6000)

In [ ]:
recordings = {
    'raw':rec_ap,
    'filtered+cmr': rec_preprocessed
                }

In [ ]:
# Visualize recording
si.plot_timeseries(recordings, 
                   segment_index=0, 
                   mode='map', # can change to trace
                   order_channel_by_depth=True,
                   clim=(-40, 40),
                   backend='ipywidgets')


### Compute necessary files for peaks/noise visualization

In [ ]:
# Noise unscaled
noise_file = output_path / 'noise_levels_unscaled.npy'
if noise_file.exists():
    noise_levels = np.load(noise_file)
else:
    noise_levels = si.get_noise_levels(rec_preprocessed, return_scaled=False)
    np.save(noise_file, noise_levels)
    
# Noise scaled
noise_file = output_path / 'noise_levels_scaled.npy'
if noise_file.exists():
    noise_levels_scaled = np.load(noise_file)
else:
    noise_levels_scaled = si.get_noise_levels(rec_preprocessed, return_scaled=True)
    np.save(noise_file, noise_levels)

In [ ]:
# Peaks
peaks_file = output_path / 'peaks.npy'
if peaks_file.exists():
    peaks = np.load(peaks_file)
else:
    peaks = detect_peaks(rec_preprocessed, noise_levels=noise_levels, **peak_detection_params, **job_kwargs)
    np.save(peaks_file, peaks)

In [ ]:
# Peak location
location_file = output_path / 'peak_locations.npy'
if location_file.exists():
    peak_locations = np.load(location_file)
else:
    peak_locations = localize_peaks(rec_preprocessed, peaks, **peak_location_params, **job_kwargs)
    np.save(location_file, peak_locations)

In [ ]:
# Visualize potential drift
widgets.interact(plot_drift,
                 time_range=widgets.FloatRangeSlider(min=0, max=rec_preprocessed.get_total_duration(), step=1),
                 peaks=widgets.fixed(peaks), 
                 rec_preprocess=widgets.fixed(rec_preprocessed), 
                 peak_locations=widgets.fixed(peak_locations), 
                 name=widgets.fixed(rec_name)                 
                )

In [ ]:
# Visualize peaks on the probe
widgets.interact(plot_peaks_axis,
                 time_range=widgets.FloatRangeSlider(min=0, max=rec_preprocessed.get_total_duration(), step=1),
                 peaks=widgets.fixed(peaks), 
                 rec_preprocess=widgets.fixed(rec_preprocessed), 
                 peak_locations=widgets.fixed(peak_locations), 
                 name=widgets.fixed(rec_name)                 
                )

In [ ]:
# Visualize peak activity as a density map
widgets.interact(plot_peaks_activity,
                 time_range=widgets.FloatRangeSlider(min=0, max=rec_preprocessed.get_total_duration(), step=1),
                 peaks=widgets.fixed(peaks), 
                 rec_preprocess=widgets.fixed(rec_preprocessed), 
                 peak_locations=widgets.fixed(peak_locations), 
                 name=widgets.fixed(rec_name)                 
                )

In [ ]:
widgets.interact(plot_noise,
                 time_range=widgets.FloatRangeSlider(value=[0,rec_preprocessed.get_total_duration()], min=0, max=rec_preprocessed.get_total_duration(), step=1),
                 clim=widgets.FloatRangeSlider(value=(0,20), min=-10, max=40, step=1),
                 noise_levels_scaled=widgets.fixed(noise_levels_scaled),
                 peaks=widgets.fixed(peaks), 
                 rec_preprocess=widgets.fixed(rec_preprocessed), 
                 peak_locations=widgets.fixed(peak_locations), 
                 name=widgets.fixed(rec_name)
                 
                )

In [ ]:
# Reading all streams in the necessary way
rec_ap = si.read_spikeglx(rec_path, stream_id='imec0.ap', load_sync_channel=True)
rec_mic = si.read_spikeglx(rec_path, stream_id='nidq', load_sync_channel=True)
rec_lf = si.read_spikeglx(rec_path, stream_id='imec0.lf', load_sync_channel=True)

In [ ]:
rec_ap

In [ ]:
# # Save new files
time_range = (3600, 10800)
# rec_lf = None
# rec_mic = None
export_files(rec_ap,
             rec_lf, 
             rec_mic, 
             rec_path, 
             output_path, 
             time_range)